Read requests in one folder

In [ ]:
from pathlib import Path
import json
import pandas as pd


path_request = "../data/requests/0521_301-20220531"
requests = Path(path_request)
files = sorted(requests.iterdir(), key=lambda x: x.name)
rows = []
for file in files:
    with file.open("r", encoding="utf-8") as f:
        data = json.load(f)
        configuration = data.get("configurationName", {})
        data_list = data.get("tasks", [])
        for task in data_list:
            rows.append({
                "configuration": configuration,
                "task_id": task["id"],
                "latitude": task["address"]["latitude"],
                "longitude": task["address"]["longitude"],
                "from": task["timeWindow"]["from"],
                "until": task["timeWindow"]["till"]
            })  
df = pd.DataFrame(rows)
#df = df.drop_duplicates()
df['task_id'] = df['task_id'].astype(int)
print(df)
print(df.info())

Locaties vergelijken

In [ ]:
from geopy.distance import geodesic

# Coordinates (latitude, longitude)
coords_1 = (0.648395,0.307167)
coords_2 = (0.646775,0.308146)

# Calculate distance in various units
distance_km = geodesic(coords_1, coords_2).meters


print(f"Distance: {distance_km:.2f} m")


Unieke location id's maken

In [ ]:

from geopy.point import Point

# 1. Initialize the unique location tracking
unique_points = [] # List of Point objects
location_map = {}  # Map index to location_id
threshold_m = 1.0  # 1 meter threshold

# 2. Iterate through the DataFrame
for idx, row in df.iterrows():
    current_pt = Point(row['latitude'], row['longitude'])
    found_match = False
    
    # Check current point against established unique points
    for i, saved_pt in enumerate(unique_points):
        if geodesic(current_pt, saved_pt).meters < threshold_m:
            df.at[idx, 'location_id'] = f"LOC_{i+1:03d}"
            found_match = True
            break
    
    # If no match within 1m, create a new unique location
    if not found_match:
        unique_points.append(current_pt)
        new_id = len(unique_points)
        df.at[idx, 'location_id'] = f"LOC_{new_id:03d}"

# 3. Display results (Rows 1644/1645 and 1801/1802 will share IDs as they are identical)
print(df[['task_id', 'latitude', 'longitude', 'location_id']].tail(10))
